In [1]:
#reload the modules before running the program
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy
from scipy import signal
from scipy.fft import fftshift
from scipy.fft import fft, fftfreq

import matplotlib.pyplot as plt
import matplotlib as mpl

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import os
import math
import cv2

In [2]:
from datasets import RFData, CameraData, SignalDataset, FuseDatasets
import metrics
import utils

In [3]:
np.random.seed(0)
torch.manual_seed(0)
data_path = r"E:\OSA_project\OSA_project\Datasets\osa_new_processed"
# trial_folders = ["v_12_5", "v_13_5", "v_13_10"]
trial_folders = os.listdir(data_path)

train_idxs = np.linspace(0, 49, 50, dtype=int)
train_folders = [trial_folders[i] for i in train_idxs]
test_folders = [trial_folders[i] for i in range(len(trial_folders)) if i not in train_idxs]


thermal_file_name = "Thermal_Camera"
num_samps_oversample = 20 # per experiment, number of samples to generate
data_length = 9000
fs = 30
out_len = 1800 # sample length generated
thermal_ext = ".tiff"

dataset_thermal_train = CameraData(data_path, train_folders, thermal_file_name, num_samps_oversample, fs, data_length, out_len, thermal_ext)
dataset_thermal_test = CameraData(data_path, test_folders, thermal_file_name, num_samps_oversample, fs, data_length, out_len, thermal_ext)

samp_f=5e6
freq_slope=60.012e12
samples_per_chirp=256
num_tx = 3
num_rx = 4
radar_file_name = "FMCW_Radar.npy"
window_size = 25 # number of range bins to use

dataset_radar_train = RFData(data_path, train_folders, data_length, radar_file_name, out_len, window_size, samples_per_chirp, samp_f, freq_slope, num_samps_oversample, num_tx, num_rx, fs)
dataset_radar_test = RFData(data_path, test_folders, data_length, radar_file_name, out_len, window_size, samples_per_chirp, samp_f, freq_slope, num_samps_oversample, num_tx, num_rx, fs)

# We don't need the following code block below because the ground truth signals are saved in npy files (assuming seed is 0)

In [6]:
"""
np.random.seed(0)
torch.manual_seed(0)
vital_dict_file_name = "gt_dict.pkl"
vital_key_radar = "CHEST"
vital_key_thermal = "AIR_flow"
l_freq_bpm = 5
u_freq_bpm = 30
dataset_OSA_train = SignalDataset(data_path, train_folders, vital_dict_file_name, 'OSA', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
                              normalize=False)
dataset_CSA_train = SignalDataset(data_path, train_folders, vital_dict_file_name, 'CSA', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
                              normalize=False)
dataset_hypopnea_train = SignalDataset(data_path, train_folders, vital_dict_file_name, 'Hypopnea', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
                              normalize=False)
dataset_partial_apnea_train = SignalDataset(data_path, train_folders, vital_dict_file_name, 'Partial_Apnea', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
                              normalize=False)
dataset_gt_thermal_train = SignalDataset(data_path, train_folders, vital_dict_file_name, vital_key_thermal, data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False)
dataset_gt_radar_train = SignalDataset(data_path, train_folders, vital_dict_file_name, vital_key_radar, data_length, out_len, True, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False)
"""
# dataset_apnea_test = SignalDataset(data_path, test_folders, vital_dict_file_name, 'Sleep_Apnea', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
#                               normalize=False)
# dataset_gt_thermal_test = SignalDataset(data_path, test_folders, vital_dict_file_name, vital_key_thermal, data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False)
# dataset_gt_radar_test = SignalDataset(data_path, test_folders, vital_dict_file_name, vital_key_radar, data_length, out_len, True, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False)
# np.random.seed(0)
# torch.manual_seed(0)
# vital_dict_file_name = "gt_dict.pkl"
# vital_key_radar = "CHEST"
# vital_key_thermal = "AIR_flow"
# l_freq_bpm = 5
# u_freq_bpm = 30
# dataset_apnea_train = SignalDataset(data_path, train_folders, vital_dict_file_name, 'Sleep_Apnea', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
#                               normalize=False)
# dataset_hypopnea_train = SignalDataset(data_path, train_folders, vital_dict_file_name, 'Hypopnea', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
#                               normalize=False)
# dataset_partial_apnea_train = SignalDataset(data_path, train_folders, vital_dict_file_name, 'Partial_Apnea', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
#                               normalize=False)
# dataset_gt_thermal_train = SignalDataset(data_path, train_folders, vital_dict_file_name, vital_key_thermal, data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False)
# dataset_gt_radar_train = SignalDataset(data_path, train_folders, vital_dict_file_name, vital_key_radar, data_length, out_len, True, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False)

# # dataset_apnea_test = SignalDataset(data_path, test_folders, vital_dict_file_name, 'Sleep_Apnea', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
# #                               normalize=False)
# # dataset_gt_thermal_test = SignalDataset(data_path, test_folders, vital_dict_file_name, vital_key_thermal, data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False)
# # dataset_gt_radar_test = SignalDataset(data_path, test_folders, vital_dict_file_name, vital_key_radar, data_length, out_len, True, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False)

'\nnp.random.seed(0)\ntorch.manual_seed(0)\nvital_dict_file_name = "gt_dict.pkl"\nvital_key_radar = "CHEST"\nvital_key_thermal = "AIR_flow"\nl_freq_bpm = 5\nu_freq_bpm = 30\ndataset_OSA_train = SignalDataset(data_path, train_folders, vital_dict_file_name, \'OSA\', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,\n                              normalize=False)\ndataset_CSA_train = SignalDataset(data_path, train_folders, vital_dict_file_name, \'CSA\', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,\n                              normalize=False)\ndataset_hypopnea_train = SignalDataset(data_path, train_folders, vital_dict_file_name, \'Hypopnea\', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,\n                              normalize=False)\ndataset_partial_apnea_train = SignalDataset(data_path, train_folders, vital_dict_file_name, \'Parti

In [7]:
"""
np.random.seed(0)
torch.manual_seed(0)
fused_dataset_train = FuseDatasets([dataset_radar_train, dataset_thermal_train, dataset_gt_radar_train, dataset_gt_thermal_train, dataset_OSA_train, dataset_CSA_train, dataset_hypopnea_train, dataset_partial_apnea_train], ["radar", "thermal", "gt_radar", "gt_ir", "gt_OSA", "gt_CSA", "hypopnea", "partial_apnea"], out_len=out_len)
# fused_dataset_test = FuseDatasets([dataset_radar_test, dataset_thermal_test, dataset_gt_radar_test, dataset_gt_thermal_test, dataset_apnea_test], ["radar", "thermal", "gt_radar", "gt_ir", "gt_apnea"], out_len=out_len)
"""
# np.random.seed(0)
# torch.manual_seed(0)
# fused_dataset_train = FuseDatasets([dataset_radar_train, dataset_thermal_train, dataset_gt_radar_train, dataset_gt_thermal_train, dataset_apnea_train, dataset_hypopnea_train, dataset_partial_apnea_train], ["radar", "thermal", "gt_radar", "gt_ir", "gt_apnea", "hypopnea", "partial_apnea"], out_len=out_len)
# # fused_dataset_test = FuseDatasets([dataset_radar_test, dataset_thermal_test, dataset_gt_radar_test, dataset_gt_thermal_test, dataset_apnea_test], ["radar", "thermal", "gt_radar", "gt_ir", "gt_apnea"], out_len=out_len)

'\nnp.random.seed(0)\ntorch.manual_seed(0)\nfused_dataset_train = FuseDatasets([dataset_radar_train, dataset_thermal_train, dataset_gt_radar_train, dataset_gt_thermal_train, dataset_OSA_train, dataset_CSA_train, dataset_hypopnea_train, dataset_partial_apnea_train], ["radar", "thermal", "gt_radar", "gt_ir", "gt_OSA", "gt_CSA", "hypopnea", "partial_apnea"], out_len=out_len)\n# fused_dataset_test = FuseDatasets([dataset_radar_test, dataset_thermal_test, dataset_gt_radar_test, dataset_gt_thermal_test, dataset_apnea_test], ["radar", "thermal", "gt_radar", "gt_ir", "gt_apnea"], out_len=out_len)\n'

In [8]:
save_path = r"E:\OSA_project\Emir_Clean\pre_load_datav2"

gt_OSA = np.load(os.path.join(save_path,r"gt_OSA.npy"))
gt_CSA  = np.load(os.path.join(save_path,r"gt_CSA.npy"))
gt_hypopnea = np.load(os.path.join(save_path,r"gt_hypopnea.npy"))
gt_radar = np.load(os.path.join(save_path,r"gt_radar_arr.npy"))
gt_ir = np.load(os.path.join(save_path,r"gt_ir_arr.npy"))

In [9]:
apnea_idx = []
for i in range(len(dataset_apnea_train)):
    batch = dataset_apnea_train[i]
    if(np.mean(batch) > 0.1):
        apnea_idx.append(i)

print(100*len(apnea_idx)/len(dataset_apnea_train))

8.2


In [8]:
root = r"E:\OSA_project\Emir_Clean\saved_idx_arrs"
NUKS_idx = np.load(os.path.join(root, "NUKS_idx.npy"))
mode_lock_idx = np.load(os.path.join(root, "mode_lock_idx.npy"))
combined_movement_idx = np.load(os.path.join(root, "combined_movement_idx.npy"))
movement_idx = np.load(os.path.join(root, "movement_new_idx.npy"))

In [9]:
root = r"E:\OSA_project\Emir_Clean\pre_load_data"
gt_apnea = np.load(os.path.join(root, "gt_apnea.npy"))
thermal_arr = np.load(os.path.join(root, "thermal_arr.npy"))
t_arr = np.linspace(0, 1800/30, 1800)

In [10]:
thermal_vid_arr = np.load(os.path.join(root, "thermal_vid_arr.npy"))

In [53]:
gt_movement_idx = np.load(r"E:\OSA_project\Emir_Clean\saved_idx_arrs\gt_movement_idx.npy")
discard_idx = np.load(r"E:\OSA_project\Emir_Clean\saved_idx_arrs\discard_idx.npy")

In [155]:
gt_arr = np.zeros(1000)
pred_arr = np.zeros(1000)
exp_movement_idx = []
# vid_idxs = []
id = 10
for i in tqdm(range(len(fused_dataset_train))):
    if(i in NUKS_idx):
        # print("NUKS")
        pass
    elif(i in mode_lock_idx):
        # print("Mode Lock")
        pass
    elif(i in discard_idx):
        # print("Discard")
        pass
    else:
        # print("Normal")
        # 3.3684210526315788, 40, 24
        if((utils.movement_detector(thermal_arr[i], dmin=dmin_arr[(best_idx[id] % 100) // 10], dmax=dmax_arr[(best_idx[id] % 100) % 10], th=th_arr[best_idx[id] // 100]) == True) or (i in vid_idxs)):
            exp_movement_idx.append(i)
        
        # if(utils.vid_movement_detector(thermal_vid_arr[i], lb=0, ub=200) == True):
        #     if(i not in exp_movement_idx):
        #         exp_movement_idx.append(i)
        #     vid_idxs.append(i)




for i in range(1000):
    if(i in gt_movement_idx):
        gt_arr[i] = 1
    if(i in exp_movement_idx):
        pred_arr[i] = 1

precision, recall, accuracy, confusion_matrix = utils.get_stats(np.array(pred_arr), np.array(gt_arr))

    

100%|██████████| 1000/1000 [00:07<00:00, 130.57it/s]


In [156]:
print(precision, recall, accuracy, confusion_matrix)

0.9532163742690059 0.9421965317919075 0.982 [[163.  10.]
 [  8. 819.]]


In [160]:
dmin_arr = np.linspace(15,40, 10, dtype=int)
dmax_arr = np.linspace(15,40, 10, dtype=int)
th_arr = np.linspace(1, 8, 20)

prods = []
stats = []
fps = []
for th in tqdm(th_arr):
    for dmin in tqdm(dmin_arr):
        for dmax in dmax_arr:
            gt_arr = np.zeros(1000)
            pred_arr = np.zeros(1000)
            exp_movement_idx = []
            # exp_movement_idx.extend(vid_idxs)

            for i in range(1000):
                if(i in NUKS_idx):
                    # print("NUKS")
                    pass
                elif(i in mode_lock_idx):
                    # print("Mode Lock")
                    pass
                elif(i in discard_idx):
                    # print("Discard")
                    pass
                elif(i not in vid_idxs):
                    # print("Normal")
                    if(utils.movement_detector(thermal_arr[i], dmin=dmin, dmax=dmax, th=th) == True):
                        exp_movement_idx.append(i)
            
            for i in range(1000):
                if(i in gt_movement_idx):
                    gt_arr[i] = 1
                if((i in exp_movement_idx) or (i in vid_idxs)):
                    pred_arr[i] = 1

            precision, recall, accuracy, confusion_matrix = utils.get_stats(np.array(pred_arr), np.array(gt_arr))
            prods.append(precision*recall)
            fps.append(confusion_matrix[1,0]) # want to minimize
            stats.append([precision, recall, accuracy, confusion_matrix])


  0%|          | 0/20 [00:00<?, ?it/s]

In [71]:
print(precision, recall, accuracy, confusion_matrix)

0.9640718562874252 0.930635838150289 0.982 [[161.  12.]
 [  6. 821.]]


In [ ]:
# if array contains nan set that entry to zero
for i in range(len(prods)):
    if(math.isnan(prods[i])):
        prods[i] = 0

In [ ]:
best_idx = np.argsort(prods)[::-1]
best_idx = np.array(best_idx, dtype=int)

In [ ]:
id = 0
th_arr[best_idx[id] // 100], dmin_arr[(best_idx[id] % 100) // 10], dmax_arr[(best_idx[id] % 100) % 10]
# (3.3684210526315788, 40, 24)

In [133]:
print(prods[best_idx[0]], stats[best_idx[0]])

0.8640753175729466 [0.9461077844311377, 0.9132947976878613, 0.976, array([[158.,  15.],
       [  9., 818.]])]


In [55]:
#else
print(precision, recall, accuracy, confusion_matrix)    

[[146.  27.]
 [  6. 821.]]


In [67]:
print(precision, recall, accuracy, confusion_matrix)    

0.2208994708994709 0.9653179190751445 0.405 [[167.   6.]
 [589. 238.]]


In [59]:
for i in range(len(discard_idx)):
    index = discard_idx[i]
    image_arr = fused_dataset_train[index]["thermal"][0,:,:,:]
    # thumbnail = f"E:\OSA_project\Kai_data\{index}.png"
    # plt.imshow(image_arr[0])
    # plt.savefig(thumbnail) 
    norm_im = utils.normalize(image_arr)
    # Output video file name
    output_video = f"E:\OSA_project\{index}.mp4"
    height, width, = 64, 64
    fs = 30

    # Create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'MP42')
    video = cv2.VideoWriter(output_video, fourcc, fs, (width, height))

    # Iterate over the image files and write each frame to the video
    for i in range(len(norm_im)):
        img = cv2.applyColorMap(norm_im[i], cv2.COLORMAP_JET)
        video.write(img)

    # Release the VideoWriter object
    video.release()


In [ ]:
for idx in range(900,1000):
    if((idx not in NUKS_idx) and (idx not in mode_lock_idx)):
        diff_arr = np.diff(thermal_vid_arr[idx], axis=0)
        mean_arr = np.std(diff_arr**2, axis=(1,2))
        pred = np.mean(utils.predict(thermal_arr[idx], 29, 31, 0.3394, plot=True)) > 0.1
        print(idx, np.std(mean_arr), pred)


        # mean_arr = mean_arr/np.std(mean_arr)
        # lmin, lmax = utils.hl_envelopes_idx(mean_arr, dmin=29, dmax=31)
        # utils.plot_envelope(t_arr[:-1], mean_arr, lmin, lmax)

        # plt.plot(t_arr[:-1] , mean_arr)
        # plt.show()
        plt.plot(t_arr, gt_apnea[idx])
        plt.show()

In [11]:
len(movement_idx), len(combined_movement_idx)

(176, 204)

In [8]:
count = 0

for i in combined_movement_idx:
    if i not in movement_idx:
        count += 1

print(count)

56


In [9]:
count = 0

for i in movement_idx:
    if i not in combined_movement_idx:
        count += 1

print(count)

28


In [ ]:
# Next steps
# 1.1 Re-run grid search for thermal, save optimal parameters and finalize stats w/o motion--> need to wait for motion filtering algorithm (IN PROGRESS)
# 1.2 create an algorithm that handles motion --> do this after finishing 1.1

# 2.1 create a function that splits indexes based on patient number for AHI metrics -->  DONE

# 3.1 make an algorithm for radar and run preliminary results --> can be done immediately

# 4.1 create an algorithm that distinguishes between central apnea and obstructive apnea --> waiting for Kai to provide data and waiting to finish radar algorithm